In [ ]:
# Third-party imports
%matplotlib inline
import mxnet as mx
from mxnet import gluon
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
pd.plotting.register_matplotlib_converters()

# Download and read data

In [ ]:
from gjopen.utilities import df_tools, read_files 
df = read_files.download_and_read_noaa_txt("ftp://aftp.cmdl.noaa.gov/products/trends/co2/co2_trend_gl.txt")
df_datetime_indexed = df_tools.set_noaa_datetime_index(df)

In [ ]:
# for i in range(365):
#     data_all = df_tools.add_a_day(data_all)

In [ ]:
df_datetime_indexed

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

def estimate_holt_winters(data, freq, data_length_in_plot=0, prediction_length=12, 
                          y_label="Y", prediction_intervals=None, target_date=None):
    """
    Prediction made with Holt winters, aka ETS triple 
    param data: pandas Series with datetime indexing
    """
    model = ExponentialSmoothing(data, trend='mul', seasonal='add', damped=False)

    model_fit = model.fit()

    # ~~~
    # Predict
    # ~~~

    pred_ets = model_fit.forecast(prediction_length)
    print(pred_ets)

    ax = data[-data_length_in_plot:].plot(label='observed', figsize=(14, 7))
    pred_ets.plot(ax=ax, label='Forecast')
    ax.set_xlabel('Date')
    ax.set_ylabel(y_label)
    plt.legend()
    plt.show()
    

In [ ]:
from gluonts.model.prophet import ProphetPredictor
from gluonts.dataset.util import to_pandas

def estimate_prophet(data, freq, data_length_in_plot=0, prediction_length=12, 
                     y_label="Y", prediction_intervals=[90.0, 100.0], target_date=None):
    """
    TODO: prophet_params [dict], init_model [func]
    """
    prophet_predictor = ProphetPredictor(
        freq=freq, 
        prediction_length=prediction_length, 
    )
    legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    for test_entry, forecast in zip(data, prophet_predictor.predict(data)):
        to_pandas(test_entry)[-data_length_in_plot:].plot(linewidth=2)
        forecast.plot(color='g', prediction_intervals=prediction_intervals)
        if target_date is not None:
            print(forecast.index)
            print(forecast.samples.shape)#[target_date]
    ax.set_xlabel('Date')
    ax.set_ylabel(y_label)
    plt.legend(legend, loc="best")
    plt.grid(which='both')

In [ ]:
def predict(data, freq, predict_func, use_gluonts=False, series_key=None,
            data_length_in_plot=0, prediction_length=12, y_label="Y",
            prediction_intervals=[90, 95, 99], target_date=None):
    """
    TODO: add explanation
    """
    data = data.resample(freq).apply(lambda ser: ser.iloc[-1,])
    
    if use_gluonts:
        # Assume using prophet predictor
        data = ListDataset(
            [{"start": data.index[0], "target": data[series_key]}], # TODO: changed from .cycle
            freq = freq
        )
    else: # TODO: if using some other models, work on this
        data = data[series_key]
    
    predict_func(data, freq, data_length_in_plot, prediction_length, y_label, prediction_intervals, target_date)
        
    
predict(
    df_datetime_indexed, 
    "MS", 
    estimate_holt_winters, 
    series_key="cycle", 
    data_length_in_plot=24, 
    prediction_length=24, 
    y_label="CO2"
)

In [ ]:
predict(
    df_datetime_indexed, 
    "MS", 
    estimate_prophet, 
    use_gluonts=True, 
    data_length_in_plot=24, 
    prediction_length=24, 
    y_label="CO2",
    target_date="2020-12-01 00:00:00"
)

# build dataset

In [ ]:
freq="MS"
data = data_all.resample(freq).apply(lambda ser: ser.iloc[-1,])

In [ ]:
# ~~~
# 
# ~~~

estimate_holt_winters(data['cycle'], 96, 40, y_label='CO2')

In [ ]:
import mxnet as mx
ctx = mx.Context("gpu")
mx.context.num_gpus()

In [ ]:

from gluonts.dataset.common import ListDataset
training_data = ListDataset(
    [{"start": data.index[0], "target": data.cycle[:"2019-06-30 00:00:00"]}], # TODO: index in programatical way
    freq = freq
)
validation_data = ListDataset(
    [{"start": data.index[0], "target": data.cycle[:"2020-01-30 00:00:00"]}], # TODO: index in programatical way
    freq = freq
)
test_data = ListDataset(
    [{"start": data.index[0], "target": data.cycle}],
    freq = freq
)

In [ ]:

from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.npts import NPTSPredictor
from gluonts.model.transformer import TransformerEstimator
from gluonts.trainer import Trainer

estimator = DeepFactorEstimator(
#     num_hidden_dimensions=[100],
#     context_length=24,
    num_hidden_global=10,
    num_layers_global=1,
    num_factors=5,
    num_hidden_local=5,
    num_layers_local=1,
    embedding_dimension=5,
    freq=freq, 
    prediction_length=24, 
    trainer=Trainer(
        epochs=10, 
        ctx = ctx,
        learning_rate=3e-3, 
        num_batches_per_epoch=10
    ),
)
predictor = estimator.train(
    training_data=training_data,
    validation_data=validation_data
)



In [ ]:

from gluonts.dataset.util import to_pandas

for test_entry, forecast in zip(test_data, predictor.predict(test_data)):
    to_pandas(test_entry).plot(linewidth=2)
    forecast.plot(color='g', prediction_intervals=[50.0, 90.0])
plt.grid(which='both')